In [41]:
import allel
import zarr
import numcodecs
import numpy as np
import sys
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import itertools

In [42]:
vcf_path = '/Users/ryanmckeown/Documents/andersen_lab/data_exploration/tajima/III.vcf.gz'
hdf5_path = '/Users/ryanmckeown/Documents/andersen_lab/data_exploration/III.h5'

In [43]:
allel.vcf_to_hdf5(vcf_path, hdf5_path, fields='*', overwrite=True)

In [44]:
callset = h5py.File(hdf5_path, mode='r')

In [45]:
# initialize stuff
chrom = callset['variants/CHROM']
pos = callset['variants/POS']
gt = callset['calldata/GT']
g = allel.GenotypeArray(gt[:, :])
ac = g.count_alleles()

In [46]:
# pi
pi, windows, n_bases, counts = allel.windowed_diversity(pos, ac, size=1000, start = 1)
# theta
theta_hat_w, windows, n_bases, counts = allel.windowed_watterson_theta(pos, ac, size=1000, start = 1)
# tajima's
D, windows, counts = allel.windowed_tajima_d(pos, ac, size=1000, min_sites=3, start = 1)
# mean of windows for each statistic
x = np.asarray(windows).mean(axis=1)

In [47]:
# output data - whole population

In [48]:
pd_output = pd.DataFrame({'chrom':"III",
              'start':windows[0:,0],
              'end':windows[0:,1],
              'pi':pi[0:],
              'theta':theta_hat_w[0:],
              'td':D[0:]})

In [49]:
pd_output.to_csv(path_or_buf="chromIII_td.csv")

In [50]:
# sliding window

In [51]:
# pi
pi, windows, n_bases, counts = allel.windowed_diversity(pos, ac, size=10000, step=1000, start = 1)
# theta
theta_hat_w, windows, n_bases, counts = allel.windowed_watterson_theta(pos, ac, size=10000, step=1000, start = 1)
# tajima's
D, windows, counts = allel.windowed_tajima_d(pos, ac, size=10000, step=1000, start = 1, min_sites=3)
# mean of windows for each statistic
x = np.asarray(windows).mean(axis=1)

In [52]:
pd_output = pd.DataFrame({'chrom':"III",
              'start':windows[0:,0],
              'end':windows[0:,1],
              'pi':pi[0:],
              'theta':theta_hat_w[0:],
              'td':D[0:]})

In [53]:
pd_output.to_csv(path_or_buf="chromIII_td_slide.csv")